<div style="background-color:#000;"><img src="pqn.png"></img></div>

### Download historical price data for selected stocks

We start by downloading the historical stock price data for a set of companies. We use Yahoo Finance to get the data.

In [ ]:
import pandas as pd
import yfinance as yf

from pypfopt import expected_returns, risk_models, black_litterman
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier

import warnings
warnings.filterwarnings("ignore")

In [ ]:
mag_7 = [
    "AAPL",
    "AMZN",
    "NVDA",
    "TLSA",
    "GOOGL",
    "META",
    "MSFT",
]

In [ ]:
prices = yf.download(mag_7, start="2020-01-01")["Adj Close"]

We define a list of major tech companies, often referred to as the "Magnificent 7". We then use the yfinance library to download the adjusted closing price of these stocks starting from January 1, 2020. This data will be used to calculate returns and risks for our portfolio optimization. The adjusted closing price accounts for any corporate actions like splits and dividends, providing a more accurate reflection of an investor’s return.

### Define our views and market capitalizations

Next, we define our own views on the expected returns for each stock and gather market capitalization data.

In [ ]:
views = {
    "AAPL": 0.05,
    "AMZN": 0.15,
    "NVDA": 0.25,
    "TLSA": -0.05,
    "GOOGL": -0.15,
    "META": 0.07,
    "MSFT": 0.12
}

In [ ]:
mcaps = {
    "AAPL": 2.5e12,
    "AMZN": 1.3e12,
    "NVDA": 1.0e12,
    "TLSA": 0.9e12,
    "GOOGL": 1.4e12,
    "META": 0.7e12,
    "MSFT": 2.0e12,
}

We define our views on the expected returns for each stock. These views represent our subjective beliefs about the future performance of these companies. For example, we expect Apple's return to be 5% and Amazon's to be 15%. Additionally, we provide the market capitalizations of each company, which reflect their total market value. This information will be used in the Black-Litterman model to incorporate market views.

### Calculate expected returns and covariance

We calculate historical expected returns and the covariance matrix to assess the risk and return of our portfolio.

In [ ]:
expected_returns_mean = expected_returns.mean_historical_return(prices)
cov_matrix = risk_models.sample_cov(prices)

In [ ]:
delta = black_litterman.market_implied_risk_aversion(prices)
market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, cov_matrix)

We compute the historical expected returns for each company using the mean of their historical returns. The covariance matrix measures how the returns of the stocks move together, providing insight into the portfolio's risk. We also calculate the market's implied risk aversion, which reflects the market's overall attitude towards risk. With the market capitalizations and risk aversion, we estimate the market's prior expected returns, which serve as a baseline before incorporating our views.

### Create a Black-Litterman model and optimize the portfolio

We create a Black-Litterman model using our views and optimize the portfolio to maximize the Sharpe ratio.

In [ ]:
bl = BlackLittermanModel(
    cov_matrix,
    absolute_views=views,
    pi=market_prior
)

In [ ]:
bl_returns = bl.bl_returns()

In [ ]:
ef = EfficientFrontier(bl_returns, cov_matrix)

In [ ]:
weights = ef.max_sharpe()

In [ ]:
bl_weights = pd.DataFrame(
    list(weights.items()), 
    columns=["symbol", "weight"]
).set_index("symbol")

In [ ]:
performance = ef.portfolio_performance(verbose=True)

We create a Black-Litterman model using the covariance matrix, our views on expected returns, and the market's prior returns. This model adjusts the market's baseline expectations with our views, resulting in a set of expected returns that reflect both market data and personal insights. We then use these adjusted returns to construct an efficient frontier, which shows the best possible returns for a given level of risk. We optimize the portfolio to achieve the maximum Sharpe ratio, which balances risk and return, and derive the optimal weights for each stock. The calculated performance metrics provide an overview of the portfolio's return, volatility, and risk-adjusted return.

### Your next steps

Try experimenting with different views on the expected returns for each stock. You can adjust the values in the "views" dictionary to see how they affect the optimized portfolio weights. This exercise will help you understand the impact of subjective beliefs on portfolio construction and risk management.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.